## Pipelines: Putting your entire ML workflow together

Let's import everything we're going to use this morning, like always:

In [11]:
import sys
sys.path

['',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python36.zip',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/lib-dynload',
 '/Users/suesong/Library/Python/3.6/lib/python/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages',
 '/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/IPython/extensions',
 '/Users/suesong/.ipython']

In [5]:
#data handling, model creation/evaluation
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import scipy.stats as stats
from sklearn_pandas import DataFrameMapper, cross_val_score
from sklearn.model_selection import cross_val_score as cv_sk
# visualization
%matplotlib inline
import seaborn as sns

### Pipelines: Putting An Entire Model Together End to End

Ok, the last thing we are going to learn how to do, is how to combine every aspect of creating and using a supervised machine learning model:

1. Transforming your original data (removing skew, standard scaling, encoding categorical variables as numbers)
2. Training and validating a model on that data
3. Picking parameters for a given model to optimize accuracy/precision/recall/f1 score, etc.

Let's try to see how we would do this without a pipeline. Let's get some data:

In [6]:
columns = ["sex","length","diam","height","whole","shucked","viscera","shell","age"]
numeric_columns = columns[1:-1]
categorical_columns = columns[0]
target = columns[-1]

abalone_data = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/abalone/abalone.data",
                           names=columns)
abalone_data.head()

,sex,length,diam,height,whole,shucked,viscera,shell,age
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,15
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,7
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,7


Now let's preprocess it in the standard way I've shown you:

1. Let's convert the categorical column using one-hot encoding
2. Standard scale (Z-score) the numeric columns

In [12]:
# variable encoding 
## drop off last column because its unnecessary
X_categorical = pd.get_dummies(abalone_data[categorical_columns]).astype(int).iloc[:,:-1]

#get and transform numeric features
X_numeric = abalone_data[numeric_columns]
X_numeric[numeric_columns] = StandardScaler().fit_transform(X_numeric)

#get outcome variable
y = abalone_data[target]

#combine transformed categorical and numeric features
X_final = pd.concat((X_numeric,X_categorical),axis=1)

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3144: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:3116: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentat

In [13]:
X_final

,length,diam,height,whole,shucked,viscera,shell,F,I
0,-0.574558,-0.432149,-1.064424,-0.641898,-0.607685,-0.726212,-0.638217,0,0
1,-1.448986,-1.439929,-1.183978,-1.230277,-1.170910,-1.205221,-1.212987,0,0
2,0.050033,0.122130,-0.107991,-0.309469,-0.463500,-0.356690,-0.207139,1,0
3,-0.699476,-0.432149,-0.347099,-0.637819,-0.648238,-0.607600,-0.602294,0,0
4,-1.615544,-1.540707,-1.423087,-1.272086,-1.215968,-1.287337,-1.320757,0,1
5,-0.824395,-1.087206,-1.064424,-0.973307,-0.983919,-0.940625,-0.853756,0,1
6,0.050033,0.071741,0.250672,-0.104505,-0.551363,-0.356690,0.655017,1,0
7,0.174951,0.172519,-0.347099,-0.123880,-0.294533,-0.283698,0.152092,1,0
8,-0.408000,-0.381760,-0.347099,-0.651076,-0.643732,-0.621286,-0.530447,0,0
9,0.216591,0.323686,0.250672,0.134109,-0.202164,-0.270012,0.583170,1,0


And now, let's do some standard 10-fold cross-validation scoring:

In [14]:
#create rf regressor and check 10-fold RMSE
rf = RandomForestRegressor()

cross_val_scores = np.abs(cross_val_score(rf,
                                          X_final,
                                          y,
                                          scoring = "neg_mean_squared_error", # minimizing residuals
                                          cv=10))

rmse_cross_val_scores = list(map(np.sqrt, 
                                 cross_val_scores))

print("Mean 10-fold rmse: ", str(np.mean(rmse_cross_val_scores)))
print("Std 10-fold rmse: ", str(np.std(rmse_cross_val_scores)))

Mean 10-fold rmse:  2.242919811448675
Std 10-fold rmse:  0.6206212313126896


Now, we are going to do the same thing using a combination of [sklearn-pandas](https://github.com/scikit-learn-contrib/sklearn-pandas) and Scikit-learn's pipeline feature.

In [16]:
def my_analyzer(x):
    return [x]

#convert categoricals into one-hot-encoded features
#standard scale all numeric columns
mapper = DataFrameMapper([
    ('sex', # column you want to operate on; this can be done list of columns
     CountVectorizer(analyzer = my_analyzer)), # dummy-coding the categorical variable (returning 3 columns)
    (abalone_data.columns.tolist()[1:-1], StandardScaler())]) # everything but the first column gets scaled (columns first to last)

In [17]:
mapper.fit_transform(abalone_data)

array([[ 0.        ,  0.        ,  1.        , ..., -0.60768536,
        -0.72621157, -0.63821689],
       [ 0.        ,  0.        ,  1.        , ..., -1.17090984,
        -1.20522124, -1.21298732],
       [ 1.        ,  0.        ,  0.        , ..., -0.4634999 ,
        -0.35668983, -0.20713907],
       ...,
       [ 0.        ,  0.        ,  1.        , ...,  0.74855917,
         0.97541324,  0.49695471],
       [ 1.        ,  0.        ,  0.        , ...,  0.77334105,
         0.73362741,  0.41073914],
       [ 0.        ,  0.        ,  1.        , ...,  2.64099341,
         1.78744868,  1.84048058]])

In [18]:
from sklearn.pipeline import FeatureUnion, Pipeline

In [19]:
full_pipeline = Pipeline([("all_features", mapper),
                          ("rf_regressor", RandomForestRegressor(n_estimators=100))])

In [20]:
cv_scores = cross_val_score(full_pipeline,
                            abalone_data.iloc[:,:-1],
                            abalone_data.age,
                            cv = 10,
                            scoring = "neg_mean_squared_error",
                            n_jobs = -1,
                            verbose = 1)

[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  1.3min finished


In [21]:
np.mean(np.sqrt(np.abs(cv_scores)).round(2)) #compute RMSE

2.152

#### Exercise Time!

* Change the pipeline to perform PCA and keep only the first 6 components on the complete feature pipe (after standard scaling numeric features and encoding the categorical feature)

In [41]:
pass

Now let's do this on a slightly more involved example, where we will have to do some imputation (filling in of missing values).

Here the process will be as follows:

1. Impute missing categorical values (marked with 0 after encoding) with most frequent category using `Imputer`
2. One-hot encode the categorical columns using `OneHotEncoder`
3. Impute missing numerical values using the median value of each column using `Imputer`
4. Z-score/standardize each numerical column using `StandardScaler`
5. Combine both collections of columns (one-hot encoded categorical columns and standardized numeric columns) using `FeatureUnion`
6. Pass the whole collection to a `RandomForestClassifier` to build a Random Forest classification model.
7. Use `cross_val_score` with 10-fold cross-validation on the entire pipeline.

Ready? Let's start by loading in the data:

In [22]:
# names of the columns
kidney_columns = ["age","bp","sg","al","su","rbc","pc","pcc","ba","bgr","bu","sc","sod","pot","hemo","pcv","wc","rc","htn","dm","cad","appet","pe","ane","class"]

# importing data
kidney_data = pd.read_csv("../day_1/data/chronic_kidney_disease.csv",
                          header = None,
                          na_values = "?",
                          names = kidney_columns)
kidney_data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44.0,7800.0,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38.0,6000.0,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31.0,7500.0,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32.0,6700.0,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35.0,7300.0,4.6,no,no,no,good,no,no,ckd


Let's rearrange the columns so that the numeric columns are together, followed by all of the categorical columns:

In [23]:
# rearrange kidney columns as before
kidney_columns = kidney_columns[:5] + kidney_columns[9:18] + kidney_columns[5:9] + kidney_columns[18:]

kidney_data = kidney_data[kidney_columns]
kidney_data.head()

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,pc,pcc,ba,htn,dm,cad,appet,pe,ane,class
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,NaN,NaN,...,normal,notpresent,notpresent,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,18.0,0.8,NaN,NaN,...,normal,notpresent,notpresent,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,NaN,NaN,...,normal,notpresent,notpresent,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,abnormal,present,notpresent,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,NaN,NaN,...,normal,notpresent,notpresent,no,no,no,good,no,no,ckd


In [24]:
kidney_data.dtypes

age      float64
bp       float64
sg       float64
al       float64
su       float64
bgr      float64
bu       float64
sc       float64
sod      float64
pot      float64
hemo     float64
pcv      float64
wc       float64
rc       float64
rbc       object
pc        object
pcc       object
ba        object
htn       object
dm        object
cad       object
appet     object
pe        object
ane       object
class     object
dtype: object

In [25]:
# categorical variables
cat_cols = kidney_data.columns.tolist()[14:-1]

# numerical variables
numeric_cols = kidney_data.columns.tolist()[:14]

In [26]:
# replacing _yes into "yes"
kidney_data[cat_cols]=kidney_data[cat_cols].replace({' yes':"yes"})

In [27]:
# transformer diy for any pipeline functions
from sklearn.base import BaseEstimator, TransformerMixin

# subselect distinct columns in the dataframe 
class ItemSelector(BaseEstimator, TransformerMixin):
    """For data grouped by feature, select subset of data at a provided key.

    The data is expected to be stored in a 2D data structure, where the first
    index is over features and the second is over samples.  i.e.

    >> len(data[key]) == n_samples

    Please note that this is the opposite convention to scikit-learn feature
    matrixes (where the first index corresponds to sample).

    ItemSelector only requires that the collection implement getitem
    (data[key]).  Examples include: a dict of lists, 2D numpy array, Pandas
    DataFrame, numpy record array, etc.

    >> data = {'a': [1, 5, 2, 5, 2, 8],
               'b': [9, 4, 1, 4, 1, 3]}
    >> ds = ItemSelector(key='a')
    >> data['a'] == ds.transform(data)

    ItemSelector is not designed to handle data grouped by sample.  (e.g. a
    list of dicts).  If your data is structured this way, consider a
    transformer along the lines of `sklearn.feature_extraction.DictVectorizer`.

    Parameters
    ----------
    key : hashable, required
        The key corresponding to the desired value in a mappable.
    """
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[self.key]

In [28]:
X,y = kidney_data.iloc[:,:-1], LabelEncoder().fit_transform(kidney_data.iloc[:,-1])

And here is the code for the entire pipeline:

In [41]:
from sklearn.preprocessing import Imputer

X_transformed_pipe = FeatureUnion(
        transformer_list=[
            # Pipeline for filling in missing values, one hot encoding all categorical columns
            ('categoricals_1', Pipeline([
                ('selector', ItemSelector(key = cat_cols)), # select categorical variables 
                ('imputer', Imputer(strategy = "most_frequent")), # choose the mode
                ('encoder', OneHotEncoder(sparse = False))                    
            ])),
            # Pipeline for pulling out numeric features, filling in missing values, and scaling them
            ('numeric', Pipeline([
                ('selector', ItemSelector(key = numeric_cols)), # select numerical variables
                ('imputer', Imputer(strategy = "median")), # choose the median
                ('scaler', StandardScaler()), # impute 
            ]))])

full_pipeline = Pipeline([("all_features",X_transformed_pipe), # feature union above
                          ("rf_classifier",RandomForestClassifier())]) # classification pipeline

In [34]:
cv_sk(full_pipeline,
      X,
      y,
      cv = 10)

ValueError: could not convert string to float: 'no'

Each pipeline object contains a sequence of steps, which are stored in a list. Each step is a tuple, where the first element is the name you gave the given step, and the second element is the transformation or model you are applying at that step:

In [35]:
my_feature_union = full_pipeline.steps[0][1]

In [36]:
my_feature_union.transformer_list[1][1].steps

[('selector',
  ItemSelector(key=['age', 'bp', 'sg', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc'])),
 ('imputer',
  Imputer(axis=0, copy=True, missing_values='NaN', strategy='median', verbose=0)),
 ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]

Let's take a look at a few steps:

In [42]:
print("The first step in the pipeline:\n",full_pipeline.steps[0])
print()
print("The second step in the pipeline: \n", full_pipeline.steps[1])
print()
print("The second step's transformation/model:\n", full_pipeline.steps[1][1])
# print("Since we know this is a random forest model, lets try to get the models feature importances:\n",full_pipeline.steps[1][1].feature_importances_)

The first step in the pipeline:
 ('all_features', FeatureUnion(n_jobs=1,
       transformer_list=[('categoricals_1', Pipeline(memory=None,
     steps=[('selector', ItemSelector(key=['rbc', 'pc', 'pcc', 'ba', 'htn', 'dm', 'cad', 'appet', 'pe', 'ane'])), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='most_frequent',
    verbose=0)), ('encoder', OneHotEncod...tegy='median', verbose=0)), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))]))],
       transformer_weights=None))

The second step in the pipeline: 
 ('rf_classifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))

The 

Remember, in order to be able to get feature importances or coefficients of a given model, it needs to be trained first. Just like any other transformation in sklearn, you can fit a pipeline by calling its `fit` method:

In [38]:
# using pipeline to predict the new data
full_pipeline.fit(X,y)

ValueError: could not convert string to float: 'no'

In [39]:
full_pipeline.predict(X)

NotFittedError: This Imputer instance is not fitted yet. Call 'fit' with appropriate arguments before using this method.

Now that it's been fit, we can extract the feature importances as we wanted:

In [171]:
full_pipeline.steps[1][1].feature_importances_.round(3)

array([0.005, 0.003, 0.011, 0.004, 0.   , 0.   , 0.   , 0.   , 0.043,
       0.006, 0.001, 0.027, 0.   , 0.   , 0.001, 0.   , 0.017, 0.005,
       0.002, 0.   , 0.001, 0.01 , 0.058, 0.084, 0.002, 0.031, 0.004,
       0.086, 0.011, 0.006, 0.286, 0.139, 0.013, 0.144])

What's really great about pipelines is that you can even put them into `GridSearchCV` methods, and search across parameters to tune your models. To do so, create a dictionary entry in `param_grid` that names the step (which you named earlier) and parameters you want to test:

In [47]:
# using GridSearchCV with Pipeline, must impute first otherwise, will throw error. this will change in next sklearn release
X[cat_cols] = Imputer(strategy = "most_frequent").fit_transform(X[cat_cols])
X[numeric_cols] = Imputer(strategy="median").fit_transform(X[numeric_cols])

ValueError: could not convert string to float: 'no'

In [62]:
# for categorical values
def get_mode(my_column):
    return my_column.value_counts().index[0]

mode_per_column = X[cat_cols].apply(get_mode, axis = 0)

In [63]:
X[cat_cols].isnull().sum()

rbc      152
pc        65
pcc        4
ba         4
htn        2
dm         2
cad        2
appet      1
pe         1
ane        1
dtype: int64

In [64]:
median_per_columns = X[numeric_cols].apply(lambda x: x.median(), axis = 0)

In [65]:
X[numeric_cols] = X[numeric_cols].fillna(median_per_columns, axis = 0)

In [66]:
X

,age,bp,sg,al,su,bgr,bu,sc,sod,pot,...,rbc,pc,pcc,ba,htn,dm,cad,appet,pe,ane
0,48.0,80.0,1.020,1.0,0.0,121.0,36.0,1.2,138.0,4.4,...,NaN,normal,notpresent,notpresent,yes,yes,no,good,no,no
1,7.0,50.0,1.020,4.0,0.0,121.0,18.0,0.8,138.0,4.4,...,NaN,normal,notpresent,notpresent,no,no,no,good,no,no
2,62.0,80.0,1.010,2.0,3.0,423.0,53.0,1.8,138.0,4.4,...,normal,normal,notpresent,notpresent,no,yes,no,poor,no,yes
3,48.0,70.0,1.005,4.0,0.0,117.0,56.0,3.8,111.0,2.5,...,normal,abnormal,present,notpresent,yes,no,no,poor,yes,yes
4,51.0,80.0,1.010,2.0,0.0,106.0,26.0,1.4,138.0,4.4,...,normal,normal,notpresent,notpresent,no,no,no,good,no,no
5,60.0,90.0,1.015,3.0,0.0,74.0,25.0,1.1,142.0,3.2,...,NaN,NaN,notpresent,notpresent,yes,yes,no,good,yes,no
6,68.0,70.0,1.010,0.0,0.0,100.0,54.0,24.0,104.0,4.0,...,NaN,normal,notpresent,notpresent,no,no,no,good,no,no
7,24.0,80.0,1.015,2.0,4.0,410.0,31.0,1.1,138.0,4.4,...,normal,abnormal,notpresent,notpresent,no,yes,no,good,yes,no
8,52.0,100.0,1.015,3.0,0.0,138.0,60.0,1.9,138.0,4.4,...,normal,abnormal,present,notpresent,yes,yes,no,good,no,yes
9,53.0,90.0,1.020,2.0,0.0,70.0,107.0,7.2,114.0,3.7,...,abnormal,abnormal,present,notpresent,yes,yes,no,poor,no,yes


In [72]:
X_no_imputation_transformed_pipe = FeatureUnion(
        transformer_list=[
            # Pipeline for one hot encoding all categorical columns
            ('categoricals_1', Pipeline([
                ('selector', ItemSelector(key=cat_cols)),
                ('encoder', OneHotEncoder(sparse=False))                    
            ])),
            # Pipeline for pulling out numeric features and scaling them
            ('numeric', Pipeline([
                ('selector', ItemSelector(key=numeric_cols)),
                ('scaler', StandardScaler()),
            ]))])

full_pipeline = Pipeline([("all_features",X_no_imputation_transformed_pipe),
                          ("rf_classifier",RandomForestClassifier())])

In [70]:
full_pipeline.fit(X, y)

ValueError: could not convert string to float: 'no'

In [73]:
# using GridSearchCV with Pipeline, must impute first otherwise, will throw error.
from sklearn.model_selection import GridSearchCV


estimators_range = [20,50,100]
param_grid = dict(rf_classifier__n_estimators = estimators_range)

grid = GridSearchCV(full_pipeline, 
                    param_grid, 
                    cv = 20, 
                    scoring = 'accuracy',
                    n_jobs=-1)
grid.fit(X, y)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...b/python3.6/site-packages/ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
/L

In [ ]:
## n_estimator = dictates how many branches 
## estimator_range 

param_grid = dict(rf_classifier__n_estimator = estimators_range)

In [201]:
print("Best cross-validated accuracy: ",grid.best_score_)
print()
print("Best parameter found:\n",grid.best_params_)
print()
print("Fitted_model:\n",grid.best_estimator_.steps[1][1])

Best cross-validated accuracy:  0.995

Best parameter found:
 {'rf_classifier__n_estimators': 20}

Fitted_model:
 RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=20, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)


And, you can predict directly with fitted grid-searched `pipeline`, as well!

In [202]:
grid.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

#### Exercise Time!

* add a PCA transformation step before training the classifier
* search over the number of PCA components to keep using `GridSearchCV` (test whether to keep the first 5,10, or all components)